In [5]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [6]:
df = pd.read_csv('fbdh1.csv')

def label_fix(label):
    if label < -10:
        return 0
    elif label > 10:
        return 1
    else:
        return 2

df['Class'] = df['Flow'].apply(label_fix)

X = df.drop('Flow', axis = 1)
X.drop('Class', axis = 1, inplace = True)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(X)
scaled_features = scaler.transform(X)
X = pd.DataFrame(scaled_features, columns = X.columns[:])

y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
print(X_train.shape, X_test.shape)

(4664, 8) (2000, 8)


In [13]:
D1 = tf.feature_column.numeric_column('D1')
D2 = tf.feature_column.numeric_column('D2')
P1 = tf.feature_column.numeric_column('P1')
P2 = tf.feature_column.numeric_column('P2')
dD1 = tf.feature_column.numeric_column('dD1')
dD2 = tf.feature_column.numeric_column('dD2')
dP1 = tf.feature_column.numeric_column('dP1')
dP2 = tf.feature_column.numeric_column('dP2')

feat_cols = [D1, D2, P1, P2, dD1, dD2, dP1, dP2]

In [14]:
model = tf.estimator.LinearClassifier(
n_classes = 3, 
model_dir = "ongoing/train", 
feature_columns = feat_cols)

In [22]:
feats = ['D1', 'D2', 'P1', 'P2', 'dD1', 'dD2', 'dP1', 'dP2']
label = 'label'

def get_input_fn(data_set, labels, num_epochs = None, n_batch = 128, shuffle = True):
    return tf.estimator.inputs.pandas_input_fn(
       x = pd.DataFrame({k: data_set[k].values for k in feats}),
       y = pd.Series(labels.values),
       batch_size = n_batch,   
       num_epochs = num_epochs,
       shuffle = shuffle)

In [23]:
model.train(input_fn = get_input_fn(X_train, y_train,
                                      num_epochs = None,
                                      n_batch = 128,
                                      shuffle = False),
                                      steps = 1000)

W0904 22:05:11.371132 4506686912 deprecation.py:323] From /Users/mikefurr/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0904 22:05:11.413595 4506686912 deprecation.py:323] From /Users/mikefurr/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1066: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file utilities to get mtimes.


In [24]:
model.evaluate(input_fn=get_input_fn(X_test, y_test,
                                      num_epochs=1,
                                      n_batch = 128,
                                      shuffle=False),
                                      steps=1000)

{'accuracy': 0.8525,
 'average_loss': 0.38038734,
 'loss': 47.548416,
 'global_step': 2000}

In [35]:
feature_spec = tf.estimator.classifier_parse_example_spec(
          feat_cols,
          label_key = 'Class',
          label_dtype = tf.int64)

In [39]:
serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
    feature_spec,
    default_batch_size=None
)

In [40]:
export_dir_base = ""

model.export_savedmodel(export_dir_base, serving_input_receiver_fn,
                            strip_default_attrs = True)

W0904 22:21:11.332830 4506686912 deprecation.py:323] From <ipython-input-40-0e15a6e87f20>:4: Estimator.export_savedmodel (from tensorflow_estimator.python.estimator.estimator) is deprecated and will be removed in a future version.
Instructions for updating:
This function has been renamed, use `export_saved_model` instead.
W0904 22:21:11.604393 4506686912 deprecation.py:323] From /Users/mikefurr/anaconda3/lib/python3.7/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:145: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


b'ongoing/train/1567650071'

In [41]:
dir = '/Users/mikefurr/Documents/GitHub/BreathDetectionSystem/TensorFlow/ongoing/train/1567650071'

converter = tf.lite.TFLiteConverter.from_saved_model(dir)

W0904 22:23:04.460336 4506686912 deprecation.py:323] From /Users/mikefurr/anaconda3/lib/python3.7/site-packages/tensorflow/lite/python/convert_saved_model.py:60: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0904 22:23:04.859477 4506686912 deprecation.py:323] From /Users/mikefurr/anaconda3/lib/python3.7/site-packages/tensorflow/lite/python/util.py:238: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0904 22:23:04.860185 4506686912 deprecation.py:323] From /Users/mikefurr/anaconda3/lib/python3.7/site-pack